In [16]:
%cd project/

[Errno 2] No such file or directory: 'project/'
/home/rapids/notebooks/project


In [ ]:
%pip install -r "requirements.txt"

In [17]:
import sys
import os
import numpy as np

from torch.utils.data import DataLoader
from torch.autograd import Variable
import torch.nn as nn
import torch.nn.functional as F
import torch
import matplotlib.pyplot as plt
plt.rcParams["figure.figsize"] = (20, 12) # (w, h)

from sklearn.model_selection import train_test_split

from random import sample
import pickle


TRAINING DATA

In [18]:
dir = "./matrices/adj/dia/"
arrays = []
for filename in os.listdir(dir):
    if filename.endswith('.npy'):
        arrays.append(np.load(dir + "/"+filename))

v = arrays
img_size = v[0].shape[1]
v_train, v_test = train_test_split(v, test_size=0.2, random_state=42)
v = v_train
print(len(v_train), len(v_test), len(v))

cuda = True if torch.cuda.is_available() else False
print(cuda)

1380 346 1380
True


In [19]:
n_epochs = 6000
batch_size = 345
# batch_size = 146
# n_epochs = 3000
# batch_size = 128

lr = 0.002
b1 = 0.5
b2 = 0.999
# n_cpu = 8
latent_dim = 100
channels = 1
# sample_interval = 400


In [20]:
class VectorialDataset(torch.utils.data.Dataset):
    def __init__(self, input_data, transform=None):
        super(VectorialDataset, self).__init__()
        self.input_data = torch.tensor(np.expand_dims(input_data, axis = 1)).float()
        self.transform = transform

    def __len__(self):
        return self.input_data.shape[0]
    
    def __getitem__(self, idx):
        if torch.is_tensor(idx):
            idx = idx.tolist()
        sample = self.input_data[idx, :]

        if self.transform:
          sample = self.transform(sample)

        return sample 


In [21]:
training_set = VectorialDataset(input_data=v)
dataloader = torch.utils.data.DataLoader(training_set, 
                                           batch_size=batch_size, 
                                           shuffle=True)
for i, imgs in enumerate(dataloader):
  print(i, imgs.shape)

0 torch.Size([345, 1, 64, 64])
1 torch.Size([345, 1, 64, 64])
2 torch.Size([345, 1, 64, 64])
3 torch.Size([345, 1, 64, 64])


In [22]:
def weights_init_normal(m):
    classname = m.__class__.__name__
    if classname.find("Conv") != -1:
        torch.nn.init.normal_(m.weight.data, 0.0, 0.02)
    elif classname.find('Linear') != -1:
        nn.init.xavier_normal_(m.weight.data)
    elif classname.find("BatchNorm2d") != -1:
        torch.nn.init.normal_(m.weight.data, 1.0, 0.02)
        torch.nn.init.constant_(m.bias.data, 0.0)


In [23]:
class Generator(nn.Module):
    def __init__(self):
        super(Generator, self).__init__()

        self.init_size = img_size // 8
        self.l1 = nn.Sequential(nn.Linear(latent_dim, 512 * self.init_size ** 2))

        self.conv_blocks = nn.Sequential(
            nn.BatchNorm2d(512),
            nn.Upsample(scale_factor=2),  # De init_size a init_size * 2
            nn.Conv2d(512, 256, 3, stride=1, padding=1),
            nn.BatchNorm2d(256, 0.8),
            nn.LeakyReLU(0.2, inplace=True),
            nn.Dropout(0.3),  # Añadimos dropout
            nn.Upsample(scale_factor=2),  # De init_size * 2 a init_size * 4
            nn.Conv2d(256, 128, 3, stride=1, padding=1),
            nn.BatchNorm2d(128, 0.8),
            nn.LeakyReLU(0.2, inplace=True),
            nn.Dropout(0.3),  # Añadimos dropout
            nn.Upsample(scale_factor=2),  # De init_size * 4 a init_size * 8 (img_size)
            nn.Conv2d(128, 64, 3, stride=1, padding=1),
            nn.BatchNorm2d(64, 0.8),
            nn.LeakyReLU(0.2, inplace=True),
            nn.Conv2d(64, 1, 3, stride=1, padding=1),
            nn.ReLU()  # Cambiado de Tanh a Sigmoid  # Usamos Tanh para salida entre [-1, 1]
            # También puedes probar con nn.Softplus()
        )

    def forward(self, z):
        out = self.l1(z)
        out = out.view(out.shape[0], 512, self.init_size, self.init_size)
        img = self.conv_blocks(out)
        return img



In [24]:
class Discriminator(nn.Module):
    def __init__(self):
        super(Discriminator, self).__init__()

        def discriminator_block(in_filters, out_filters, bn=True):
            block = [
                nn.utils.spectral_norm(nn.Conv2d(in_filters, out_filters, 3, 2, 1)),
                nn.LeakyReLU(0.2, inplace=True),
                nn.Dropout2d(0.3)  # Reducimos ligeramente el dropout
            ]
            if bn:
                block.append(nn.BatchNorm2d(out_filters, 0.8))
            return block

        self.model = nn.Sequential(
            *discriminator_block(channels, 16, bn=False),
            *discriminator_block(16, 32),
            *discriminator_block(32, 64),
            *discriminator_block(64, 128),
        )

        # Tamaño después del downsampling
        ds_size = img_size // 2 ** 4  # 4 capas con stride 2
        self.adv_layer = nn.Sequential(nn.Linear(128 * ds_size ** 2, 1))

    def forward(self, img):
        out = self.model(img)
        #print("imgshape" + str(img.shape))
        #print("outshape" + str(out.shape))
        out = out.view(out.shape[0], -1)
        validity = self.adv_layer(out)

        return validity


In [25]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [26]:
def compute_gradient_penalty(D, real_samples, fake_samples):
    alpha = torch.rand(real_samples.size(0), 1, 1, 1).to(device)
    interpolates = (alpha * real_samples + ((1 - alpha) * fake_samples)).requires_grad_(True)
    d_interpolates = D(interpolates)
    fake = torch.ones(real_samples.size(0), 1).to(device)
    gradients = torch.autograd.grad(
        outputs=d_interpolates,
        inputs=interpolates,
        grad_outputs=fake,
        create_graph=True,
        retain_graph=True,
    )[0]
    gradients = gradients.view(gradients.size(0), -1)
    gradient_penalty = ((gradients.norm(2, dim=1) - 1) ** 2).mean()
    return gradient_penalty

INITIALIZATION

In [12]:
# Loss function
# adversarial_loss = torch.nn.BCELoss()

# Initialize generator and discriminator
# generator = Generator()
# discriminator = Discriminator()

# if cuda:
#     generator.cuda()
#     discriminator.cuda()
#     # adversarial_loss.cuda()

# # Initialize weights
# generator.apply(weights_init_normal)
# discriminator.apply(weights_init_normal)


# # Optimizers
# optimizer_G = torch.optim.Adam(generator.parameters(), lr=lr*10, betas=(b1, b2),weight_decay=1e-4)
# optimizer_D = torch.optim.Adam(discriminator.parameters(), lr= lr, betas=( b1,  b2), weight_decay=1e-4)

# Tensor = torch.cuda.FloatTensor if cuda else torch.FloatTensor


TRAINING

In [13]:
# G_losses = []
# D_losses = []

# real_scores = np.zeros(n_epochs)
# fake_scores = np.zeros(n_epochs)

# lambda_gp = 10  # Coeficiente de penalización de gradiente
# n_critic = 5    # Número de pasos de entrenamiento del discriminador por cada paso del generador


# for epoch in range(n_epochs):
#     for i, imgs in enumerate(dataloader):

#         # Adversarial ground truths vectors
#         valid = Variable(Tensor(imgs.shape[0], 1).fill_(0.9), requires_grad=False)
#         fake = Variable(Tensor(imgs.shape[0], 1).fill_(0.1), requires_grad=False)

#         # Configure input
#         real_imgs = Variable(imgs.type(Tensor))

#         real_imgs = real_imgs + 0.05 * torch.randn_like(real_imgs)  # Añadir ruido pequeño a las imágenes reales


#         # -----------------
#         #  Train Generator
#         # -----------------

#         optimizer_G.zero_grad()

#         # Sample noise as generator input
#         z = Variable(Tensor(np.random.normal(0, 1, (imgs.shape[0],  latent_dim))))

#         # Generate a batch of images
#         gen_imgs = generator(z) # shape '[146, 128, 16, 16]' is invalid for input of size 19136512

#         # Loss measures generator's ability to fool the discriminator
#         g_loss = adversarial_loss(discriminator(gen_imgs), valid)

#         g_loss.backward()
#         optimizer_G.step()

#         # ---------------------
#         #  Train Discriminator
#         # ---------------------

#         optimizer_D.zero_grad()

#         # Measure discriminator's ability to classify real from generated samples
#         real_loss = adversarial_loss(discriminator(real_imgs), valid)
#         fake_loss = adversarial_loss(discriminator(gen_imgs.detach()), fake)
#         d_loss = (real_loss + fake_loss) / 2

#         #Accuracies
#         outputs = discriminator(real_imgs)
#         real_score = outputs
#         outputs = discriminator(gen_imgs.detach())
#         fake_score = outputs

#         d_loss.backward()
#         optimizer_D.step()

#         if epoch%10 == 0 and i == len(dataloader)-1:
#             print(
#             "[Epoch %d/%d] [Batch %d/%d] [D loss: %f] [G loss: %f]"
#             % (epoch, n_epochs, i, len(dataloader), d_loss.item(), g_loss.item())
#             )
                    
#         # Save Losses for plotting later and accuracies
#         G_losses.append(g_loss.item())
#         D_losses.append(d_loss.item())

#         real_scores[epoch] = real_scores[epoch]*(i/(i+1.)) + real_score.mean().data*(1./(i+1.))
#         fake_scores[epoch] = fake_scores[epoch]*(i/(i+1.)) + fake_score.mean().data*(1./(i+1.))

# print("end")

In [14]:
# import numpy as np
# import networkx as nx
# from scipy.stats import entropy

# def calculate_score(generator, real_adj_matrices, num_samples=100):
#     """
#     Calcula un score basado en la similitud de la distribución de grados
#     entre las redes reales y las redes generadas.

#     Args:
#         generator: El modelo generador entrenado.
#         real_adj_matrices: Lista o arreglo de matrices de adyacencia reales.
#         num_samples: Número de redes a generar para la evaluación.

#     Returns:
#         score: Un valor que indica la similitud entre las redes reales y generadas.
#     """
#     # Obtener el grado máximo de las redes reales
#     max_degree_real = get_max_degree(real_adj_matrices)
    
    
#     # Generar redes sintéticas
#     generated_adj_matrices = []
#     generator.eval()  # Poner el generador en modo evaluación
#     with torch.no_grad():
#         for _ in range(num_samples):
#             z = torch.randn(1, 100).to(device)
#             gen_adj = generator(z).cpu().numpy()
#             gen_adj = gen_adj.squeeze()
#             # Si usaste Tanh en la salida, desnormaliza
#             # gen_adj = (gen_adj + 1) / 2  # Rango [0, 1]
#             # Binarizar la matriz de adyacencia
#             gen_adj_bin = (gen_adj > 0.5).astype(int)
#             generated_adj_matrices.append(gen_adj_bin)
#     # Obtener el grado máximo de las redes generadas
#     max_degree_generated = get_max_degree(generated_adj_matrices)
    
#     # Obtener el grado máximo común
#     max_degree = max(max_degree_real, max_degree_generated)
    
#     # Obtener distribuciones de grados con el mismo tamaño
#     degree_distribution_real = get_average_degree_distribution(real_adj_matrices, max_degree)
    
#     # Obtener distribución de grados de las redes generadas
#     degree_distribution_generated = get_average_degree_distribution(generated_adj_matrices, max_degree)
    

#     # Calcular la distancia entre las distribuciones de grados
#     # Usamos la distancia de Jensen-Shannon
#     score = jensen_shannon_distance(degree_distribution_real, degree_distribution_generated)

#     # Volver el generador a modo de entrenamiento
#     generator.train()
    
#     return score

# def get_max_degree(adj_matrices):
#     """
#     Obtiene el grado máximo de una lista de matrices de adyacencia.
#     """
#     max_degree = 0
#     for adj in adj_matrices:
#         G = nx.from_numpy_array(adj)
#         degrees = [degree for node, degree in G.degree()]
#         max_degree = max(max_degree, max(degrees))
#     return max_degree


# def get_average_degree_distribution(adj_matrices, max_degree):
#     """
#     Calcula la distribución promedio de grados a partir de una lista de matrices de adyacencia.

#     Args:
#         adj_matrices: Lista o arreglo de matrices de adyacencia.

#     Returns:
#         degree_distribution: Distribución de grados normalizada.
#     """
#     degree_hist = np.zeros(max_degree + 1)
#     total_nodes = 0
#     for adj in adj_matrices:
#         G = nx.from_numpy_array(adj)
#         degrees = [degree for node, degree in G.degree()]
#         total_nodes += len(degrees)
#         for degree in degrees:
#             degree_hist[degree] += 1
#     # Normalizar la distribución
#     degree_distribution = degree_hist / total_nodes
#     return degree_distribution

# def jensen_shannon_distance(p, q):
#     """
#     Calcula la distancia de Jensen-Shannon entre dos distribuciones de probabilidad.

#     Args:
#         p: Distribución de probabilidad 1.
#         q: Distribución de probabilidad 2.

#     Returns:
#         js_distance: Distancia de Jensen-Shannon.
#     """
#     # Asegurarse de que p y q son arrays numpy y normalizados
#     p = np.array(p)
#     q = np.array(q)
#     p = p / np.sum(p)
#     q = q / np.sum(q)

#     m = 0.5 * (p + q)
#     js_distance = 0.5 * (entropy(p, m) + entropy(q, m))
#     return js_distance


In [12]:
import numpy as np
import networkx as nx
from scipy.stats import entropy

def calculate_score(generator, real_adj_matrices, num_samples=100):
    """
    Calcula un score basado en la similitud de la distribución de grados
    entre las redes reales y las redes generadas.

    Args:
        generator: El modelo generador entrenado.
        real_adj_matrices: Lista o arreglo de matrices de adyacencia reales.
        num_samples: Número de redes a generar para la evaluación.

    Returns:
        score: Un valor que indica la similitud entre las redes reales y generadas.
    """
    generator.eval()
    with torch.no_grad():
        # Generar redes sintéticas
        generated_adj_matrices = []
        for _ in range(num_samples):
            z = torch.randn(1, 100, device=device)
            gen_adj = generator(z)
            gen_adj = gen_adj.squeeze()
            # Si usaste Tanh en la salida, desnormaliza
            gen_adj = (gen_adj + 1) / 2  # Rango [0, 1]
            # Binarizar la matriz de adyacencia
            gen_adj_bin = (gen_adj > 0.5).float()
            generated_adj_matrices.append(gen_adj_bin)
    
    # Convertir lista a tensor
    generated_adj_tensor = torch.stack(generated_adj_matrices)  # Shape: (num_samples, N, N)

    # Calcular distribución de grados para las redes generadas
    degree_distribution_generated = get_degree_distribution_tensor(generated_adj_tensor)

    # Procesar las redes reales
    real_adj_tensors = [torch.tensor(adj, dtype=torch.float32, device=device) for adj in real_adj_matrices]
    real_adj_tensor = torch.stack(real_adj_tensors)  # Shape: (num_samples, N, N)

    # Calcular distribución de grados para las redes reales
    degree_distribution_real = get_degree_distribution_tensor(real_adj_tensor)

    # Asegurar que las distribuciones tengan la misma longitud
    max_length = max(len(degree_distribution_real), len(degree_distribution_generated))
    degree_distribution_real = pad_distribution(degree_distribution_real, max_length)
    degree_distribution_generated = pad_distribution(degree_distribution_generated, max_length)

    # Calcular la distancia de Jensen-Shannon en GPU
    score = jensen_shannon_distance_torch(degree_distribution_real, degree_distribution_generated)

    generator.train()

    return score.item()

def get_max_degree(adj_matrices):
    """
    Obtiene el grado máximo de una lista de matrices de adyacencia.
    """
    max_degree = 0
    for adj in adj_matrices:
        G = nx.from_numpy_array(adj)
        degrees = [degree for node, degree in G.degree()]
        max_degree = max(max_degree, max(degrees))
    return max_degree

def get_degree_distribution_tensor(adj_tensor):
    """
    Calcula la distribución promedio de grados a partir de un tensor de matrices de adyacencia.
    """
    # adj_tensor tiene forma (num_samples, N, N)
    degrees = adj_tensor.sum(dim=2)  # Sumar sobre las columnas para obtener grados de filas
    degrees = degrees.view(-1).long()  # Aplanar y convertir a enteros
    max_degree = degrees.max().item()
    degree_hist = torch.bincount(degrees, minlength=max_degree+1).float()
    # Normalizar la distribución
    degree_distribution = degree_hist / degrees.numel()
    return degree_distribution

def get_average_degree_distribution(adj_matrices, max_degree):
    """
    Calcula la distribución promedio de grados a partir de una lista de matrices de adyacencia.

    Args:
        adj_matrices: Lista o arreglo de matrices de adyacencia.

    Returns:
        degree_distribution: Distribución de grados normalizada.
    """
    degree_hist = np.zeros(max_degree + 1)
    total_nodes = 0
    for adj in adj_matrices:
        G = nx.from_numpy_array(adj)
        degrees = [degree for node, degree in G.degree()]
        total_nodes += len(degrees)
        for degree in degrees:
            degree_hist[degree] += 1
    # Normalizar la distribución
    degree_distribution = degree_hist / total_nodes
    return degree_distribution

def jensen_shannon_distance(p, q):
    """
    Calcula la distancia de Jensen-Shannon entre dos distribuciones de probabilidad.

    Args:
        p: Distribución de probabilidad 1.
        q: Distribución de probabilidad 2.

    Returns:
        js_distance: Distancia de Jensen-Shannon.
    """
    # Asegurarse de que p y q son arrays numpy y normalizados
    p = np.array(p)
    q = np.array(q)
    p = p / np.sum(p)
    q = q / np.sum(q)

    m = 0.5 * (p + q)
    js_distance = 0.5 * (entropy(p, m) + entropy(q, m))
    return js_distance

def pad_distribution(distribution, max_length):
    """
    Asegura que la distribución tenga una longitud de max_length rellenando con ceros si es necesario.
    """
    if len(distribution) < max_length:
        padding = torch.zeros(max_length - len(distribution), device=distribution.device)
        distribution = torch.cat([distribution, padding])
    return distribution

def jensen_shannon_distance_torch(p, q):
    """
    Calcula la distancia de Jensen-Shannon entre dos distribuciones de probabilidad usando PyTorch.
    """
    # Asegurarse de que p y q sumen 1
    p = p / p.sum()
    q = q / q.sum()
    m = 0.5 * (p + q)
    # Añadir una pequeña constante para evitar log(0)
    eps = 1e-8
    p = p + eps
    q = q + eps
    m = m + eps
    jsd = 0.5 * (p * (p / m).log()).sum() + 0.5 * (q * (q / m).log()).sum()
    return jsd.sqrt()

In [16]:
# Inicialización de listas para almacenar las pérdidas
'''
scores = []

lambda_gp = 10  # Coeficiente de penalización de gradiente
n_critic = 5    # Número de pasos de entrenamiento del discriminador por cada paso del generador

# for lgp in [1,5,10,15,20]:
generator = Generator()
discriminator = Discriminator()

if cuda:
    generator.cuda()
    discriminator.cuda()
    # adversarial_loss.cuda()

# Initialize weights
generator.apply(weights_init_normal)
discriminator.apply(weights_init_normal)


# Optimizers
optimizer_G = torch.optim.Adam(generator.parameters(), lr=lr, betas=(b1, b2),weight_decay=1e-4)
optimizer_D = torch.optim.Adam(discriminator.parameters(), lr= lr, betas=( b1,  b2), weight_decay=1e-4)

Tensor = torch.cuda.FloatTensor if cuda else torch.FloatTensor
losses_G = []
losses_D = []
critic_losses = []
validity_real = []
validity_fake = []

for epoch in range(n_epochs):
    for i, imgs in enumerate(dataloader):

        # Configuración de tensores
        real_imgs = imgs.to(device)
        # real_imgs = Variable(imgs.type(Tensor))
        optimizer_D.zero_grad()

        # Generación de ruido y muestras falsas
        z = torch.randn(imgs.size(0), 100).to(device)
        fake_imgs = generator(z).detach()
        fake_imgs.requires_grad = True

        # Pérdida del discriminador
        real_validity = discriminator(real_imgs)
        fake_validity = discriminator(fake_imgs)
        validity_real.append(real_validity.mean().item())
        validity_fake.append(fake_validity.mean().item())
        gradient_penalty = compute_gradient_penalty(discriminator, real_imgs.data, fake_imgs.data)
        critic_loss = -torch.mean(real_validity) + torch.mean(fake_validity)
        critic_losses.append(critic_loss.item())
        loss_D = critic_loss + lambda_gp * gradient_penalty

        loss_D.backward()
        optimizer_D.step()

        # Entrenamiento del generador cada n_critic iteraciones
        if i % n_critic == 0:
            optimizer_G.zero_grad()

            # Generación de muestras y cálculo de pérdida
            gen_imgs = generator(z)
            gen_validity = discriminator(gen_imgs)
            loss_G = -torch.mean(gen_validity)

            loss_G.backward()
            optimizer_G.step()
        
        
        # Almacenar las pérdidas al final de cada época
        losses_G.append(loss_G.item())
        losses_D.append(loss_D.item())

    # # Mostrar las pérdidas cada 10 épocas
    # if epoch % 10 == 0:
    #     # print(f"Epoch [{epoch}/{n_epochs}] | Loss D: {loss_D.item():.4f} | Loss G: {loss_G.item():.4f} | Score: {score:.4f}")
    #     print(f"Epoch [{epoch}/{n_epochs}] | Loss D: {loss_D.item():.4f} | Loss G: {loss_G.item():.4f}")
    if epoch % 10 == 0:
        score = calculate_score(generator, v_train, num_samples=100)
        scores.append(score)
        print(f"Epoch [{epoch}/{n_epochs}] | Loss D: {loss_D.item():.4f} | Loss G: {loss_G.item():.4f} | Score: {score:.4f} | Critic Loss: {critic_loss.item():.4f}")
        # Guardar modelos
        # torch.save(generator.state_dict(), f"generator_epoch_{epoch}.pth")
        # torch.save(discriminator.state_dict(), f"discriminator_epoch_{epoch}.pth")
    

# score = calculate_score(generator, v_train, num_samples=100)
# scores.append(score)
print(f"Score: {score:.4f}")
print("end")
'''

SyntaxError: '[' was never closed (1552287376.py, line 20)

PLOTS

In [27]:
def all_plots(losses_D, losses_G, scores, critic_losses):
  #Plot Losses
  plt.title("MoGAN-Training", size = 23, fontweight="bold")
  plt.plot(losses_D,label="Discriminator")
  plt.plot(losses_G,label="Generator")
  plt.xlabel("iterations", size = 23)
  plt.ylabel("loss", size = 23)
  plt.tick_params(labelsize=20)
  plt.legend(prop={'size': 21})
  # plt.savefig("./lossMoGAN_sem.pdf")  
  plt.show(block=False)

# Plot Scores
# plt.title("MoGAN: Validation values", size = 23, fontweight="bold")
# plt.plot(validity_fake, label='synthetic score')
# plt.plot(validity_real, label='real score')    
# plt.xlabel("epochs", size = 23)
# plt.ylabel("score", size = 23)
# plt.tick_params(labelsize=20)
# plt.legend(prop={'size': 18})
# # plt.savefig("./scoresMoGAN_sem.pdf")
# plt.show(block=False)

  # Plot Scores
  plt.title("MoGAN: Scores", size = 23, fontweight="bold")
  plt.plot(scores, label='synthetic score')
  # plt.plot(real_scores, label='real score')    
  plt.xlabel("epochs", size = 23)
  plt.ylabel("score", size = 23)
  plt.tick_params(labelsize=20)
  plt.legend(prop={'size': 18})
  # plt.savefig("./scoresMoGAN_sem.pdf")
  plt.show(block=False)

  # Plot Scores
  plt.title("MoGAN: Critic Loss", size = 23, fontweight="bold")
  plt.plot(critic_losses, label='critic loss')
  # plt.plot(real_scores, label='real score')    
  plt.xlabel("epochs", size = 23)
  plt.ylabel("score", size = 23)
  plt.tick_params(labelsize=20)
  plt.legend(prop={'size': 18})
  # plt.savefig("./scoresMoGAN_sem.pdf")
  plt.show(block=False)

Fake set and dump

In [28]:
def save_models(generator, discriminator, sufix):
  torch.save(generator.state_dict(), "./models/generator_"+sufix+".pth")
  torch.save(discriminator.state_dict(), "./models/discriminator_"+sufix+".pth")

def load_models(generator, discriminator, sufix):
  generator.load_state_dict(torch.load("./models/generator_"+sufix+".pth"))
  discriminator.load_state_dict(torch.load("./models/discriminator_"+sufix+".pth"))
  return generator, discriminator
  
def save_tests(generator, name_sufix):
  fake_set = []
  t = torch.tensor(np.random.normal(0, 1, (len(v_test),  latent_dim)), device='cuda') #instead of batch_size should be len(v_test), but depends on your GPU power.
  t = generator(t).detach().cpu().numpy()
  print(t.shape)

  for i in range(0, t.shape[0]):
    fake_set.append(np.rint(t[i][0]).astype(int))

  print("len of fake set", len(fake_set))


  with open( "./generations/fake_set_"+name_sufix+".txt", "wb") as fp:   #Pickling
    pickle.dump(fake_set, fp) 

  # with open("./generations/v_test_"+name_sufix+".txt", "wb") as fp:   #Pickling
  #   pickle.dump(v_test, fp) 

  # with open("./generations/v_train_"+name_sufix+".txt", "wb") as fp:   #Pickling
  #   pickle.dump(v_train, fp) 

In [32]:
fake_set

NameError: name 'fake_set' is not defined

In [29]:
# lr_combs = [(0.002, 0.0002, 5), (0.002, 0.0002, 10), (0.001, 0.0001, 10), (0.0002, 0.0002, 10)]
lr_combs = [(0.002, 0.0002, 10)]

lambda_gp = 10  # Coeficiente de penalización de gradiente
n_critic = 5    # Número de pasos de entrenamiento del discriminador por cada paso del generador
n_epochs = 6000

# for combs[2] in [5,10,15]:
for combs in lr_combs:

  # Inicialización de listas para almacenar las pérdidas
  scores = []
  losses_G = []
  losses_D = []
  critic_losses = []
  validity_real = []
  validity_fake = []
  generator = Generator()
  discriminator = Discriminator()

  if cuda:
      generator.cuda()
      discriminator.cuda()
      # adversarial_loss.cuda()

  # Initialize weights
  generator.apply(weights_init_normal)
  discriminator.apply(weights_init_normal)


  # Optimizers
  optimizer_G = torch.optim.Adam(generator.parameters(), lr=combs[0], betas=(b1, b2),weight_decay=1e-4)
  optimizer_D = torch.optim.Adam(discriminator.parameters(), lr= combs[1], betas=( b1,  b2), weight_decay=1e-4)

  Tensor = torch.cuda.FloatTensor if cuda else torch.FloatTensor

  for epoch in range(n_epochs):
      for i, imgs in enumerate(dataloader):

          # Configuración de tensores
          real_imgs = imgs.to(device)
          # real_imgs = Variable(imgs.type(Tensor))
          optimizer_D.zero_grad()

          # Generación de ruido y muestras falsas
          z = torch.randn(imgs.size(0), 100).to(device)
          fake_imgs = generator(z).detach()
          fake_imgs.requires_grad = True

          # Pérdida del discriminador
          real_validity = discriminator(real_imgs)
          fake_validity = discriminator(fake_imgs)
          validity_real.append(real_validity.mean().item())
          validity_fake.append(fake_validity.mean().item())
          gradient_penalty = compute_gradient_penalty(discriminator, real_imgs.data, fake_imgs.data)
          critic_loss = -torch.mean(real_validity) + torch.mean(fake_validity)
          critic_losses.append(critic_loss.item())
          loss_D = critic_loss + combs[2] * gradient_penalty

          loss_D.backward()
          optimizer_D.step()

          # Entrenamiento del generador cada n_critic iteraciones
          if i % n_critic == 0:
              optimizer_G.zero_grad()

              # Generación de muestras y cálculo de pérdida
              gen_imgs = generator(z)
              gen_validity = discriminator(gen_imgs)
              loss_G = -torch.mean(gen_validity)

              loss_G.backward()
              optimizer_G.step()
          
          
          # Almacenar las pérdidas al final de cada época
          losses_G.append(loss_G.item())
          losses_D.append(loss_D.item())

      # # Mostrar las pérdidas cada 10 épocas
      # if epoch % 10 == 0:
      #     # print(f"Epoch [{epoch}/{n_epochs}] | Loss D: {loss_D.item():.4f} | Loss G: {loss_G.item():.4f} | Score: {score:.4f}")
      #     print(f"Epoch [{epoch}/{n_epochs}] | Loss D: {loss_D.item():.4f} | Loss G: {loss_G.item():.4f}")
      if epoch % 10 == 0:
          score = calculate_score(generator, v_train, num_samples=100)
          scores.append(score)
          print(f"Epoch [{epoch}/{n_epochs}] | Loss D: {loss_D.item():.4f} | Loss G: {loss_G.item():.4f} | Score: {score:.4f} | Critic Loss: {critic_loss.item():.4f}")
          # Guardar modelos
          # torch.save(generator.state_dict(), f"generator_epoch_{epoch}.pth")
          # torch.save(discriminator.state_dict(), f"discriminator_epoch_{epoch}.pth")
      

  # score = calculate_score(generator, v_train, num_samples=100)
  # scores.append(score)
  print(f"Comb: {combs} | Lambda: {combs[2]} | Score: {score:.4f}")
  save_models(generator, discriminator, "dia_comb_"+str(combs[0])+"_"+str(combs[1])+"_lambda_"+str(combs[2]))
  save_tests(generator, "dia_comb_"+str(combs[0])+"_"+str(combs[1])+"_lambda_"+str(combs[2]))
  all_plots(losses_D, losses_G, scores, critic_losses)
  
print("end")

Epoch [0/6000] | Loss D: 8.2076 | Loss G: -0.0117 | Score: 0.5382 | Critic Loss: -0.7858
Epoch [10/6000] | Loss D: 1.2105 | Loss G: -3.1148 | Score: 0.7023 | Critic Loss: -6.6878
Epoch [20/6000] | Loss D: -2.8343 | Loss G: -5.0660 | Score: 0.7181 | Critic Loss: -8.9433
Epoch [30/6000] | Loss D: -0.1130 | Loss G: -7.7846 | Score: 0.7267 | Critic Loss: -5.4978
Epoch [40/6000] | Loss D: 0.5029 | Loss G: -6.5495 | Score: 0.7386 | Critic Loss: -4.4489
Epoch [50/6000] | Loss D: -3.3513 | Loss G: -4.9601 | Score: 0.7244 | Critic Loss: -6.7190
Epoch [60/6000] | Loss D: -5.0450 | Loss G: -8.5392 | Score: 0.7078 | Critic Loss: -8.9007
Epoch [70/6000] | Loss D: -5.1357 | Loss G: -12.4145 | Score: 0.7107 | Critic Loss: -7.3899
Epoch [80/6000] | Loss D: -9.0136 | Loss G: -9.5780 | Score: 0.7156 | Critic Loss: -10.4103
Epoch [90/6000] | Loss D: -10.5797 | Loss G: -5.3389 | Score: 0.7115 | Critic Loss: -11.7518
Epoch [100/6000] | Loss D: -11.6465 | Loss G: -9.7953 | Score: 0.7073 | Critic Loss: -12.3

RuntimeError: mat1 and mat2 must have the same dtype, but got Double and Float

In [41]:
t.type(torch.DoubleTensor)

tensor([[-0.7638, -2.4975, -1.4453,  ...,  1.5741, -0.1626, -2.1280],
        [ 1.3258,  0.3816,  0.5806,  ...,  0.6039,  0.2208, -3.0766],
        [ 0.8834,  0.2369,  1.1230,  ...,  0.1935,  2.2852, -0.3835],
        ...,
        [-1.2810, -0.8837, -0.5011,  ...,  0.7919,  0.7451,  1.5312],
        [ 0.0097,  0.4265, -0.4662,  ..., -0.4942, -0.5325,  1.0509],
        [ 1.7242, -0.3394, -0.3890,  ...,  0.8728, -0.7568,  1.2431]],
       dtype=torch.float64)

In [47]:
fake_set = []
t = torch.tensor(np.random.normal(0, 1, (len(v_test), latent_dim)), device='cuda').float()  # Ensure t is a float tensor
t = generator(t).detach().cpu().numpy()
print(t.shape)

for i in range(0, t.shape[0]):
  fake_set.append(np.rint(t[i][0]).astype(int))

print("len of fake set", len(fake_set))


with open( "./generations/fake_set_"+"dia_comb_"+str(combs[0])+"_"+str(combs[1])+"_lambda_"+str(combs[2])+".txt", "wb") as fp:   #Pickling
  pickle.dump(fake_set, fp) 

(346, 1, 64, 64)
len of fake set 346
